In [1]:
#!pip install webhoseio
!pip install simhash

In [2]:
import webhoseio, os
from gensim.models import KeyedVectors
import json
from simhash import Simhash, SimhashIndex
import numpy as np

In [3]:
webhoseio.config(token=os.environ['WEBHOSE_TOKEN'])
query_params = {
    "q": "organization:Apple",
    "ts": "1523748602856",
    "sort": "crawled"
}

KeyError: 'WEBHOSE_TOKEN'

### Load Word Vector Model

In [4]:
model_path = '/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 6/'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_w2v_AP    = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin.gz', True)
#model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [5]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [6]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [7]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

### Query Webhose for a set of 100 posts/feeds (be careful how many times you make this call)

In [30]:
# be careful how many times you make this call
# get the first batch
output = webhoseio.query("filterWebContent", query_params)

### Print original dataset titles only

In [8]:
#read webhose dataset
import json
json_data = open("/Users/arlenehuang/OneDrive - Columbia University/Summer 2020/APAN 5430/Assignment 6/webhose_Citigroup.json").readlines()
newsfeeds = []

for line in json_data:
    newsfeeds.append(json.loads(line))

In [9]:
feeds = []
i = 0
for feed in newsfeeds:
    feed['id'] = i
    print(feed['id'], str(feed['title']))
    i += 1
    feeds.append(feed)

0 It’s like the Wild West in the stock market with the ‘get-rich-crowd’ vs Wall St. pros — but it’s too easy to blame retail investors for ‘rampant speculation’
1 Esperion Therapeutics’ (ESPR) Buy Rating Reaffirmed at Credit Suisse Group
2 Why Citi is the latest Wall Street giant to back the new MEMX stock exchange, launching this fall
3 Centene (NYSE:CNC) Updates FY20 Earnings Guidance
4 CVS Health (NYSE:CVS) Issues FY 2020 After-Hours Earnings Guidance
5 Amkor Technology (NASDAQ:AMKR) Upgraded at BidaskClub
6 Apollo Investment (NASDAQ:AINV) Lifted to Buy at BidaskClub
7 Is Citigroup a Buy While It's Down?
8 Credit Suisse Group Reiterates GBX 4,070 Price Target for British American Tobacco Plc Ads (LON:BATS)
9 Credit Suisse Group Reiterates GBX 4,070 Price Target for British American Tobacco Plc Ads (LON:BATS)
10 JPMorgan Chase & Co. Analysts Give British American Tobacco Plc Ads (LON:BATS) a GBX 4,250 Price Target
11 Wall Street Banks Tank One Day After Fed Chair Says They’re “a Sour

935 The recovery is underway. Does that mean it's time for more stimulus? - CNN
936 CIBC Trims Enbridge ENB) Target Price to C$48.00
937 Grafton Group GFTU) Rating Reiterated by Liberum Capital
938 SIG SHI) Rating Reiterated by Liberum Capital
939 Broadcom (NASDAQ:AVGO) Lifted to B at TheStreet
940 Skyworks Solutions (NASDAQ:SWKS) Upgraded to “Buy” at BidaskClub
941 Skyworks Solutions (NASDAQ:SWKS) Upgraded to “Buy” at BidaskClub
942 GoPro Inc (NASDAQ:GPRO) Receives $4.05 Consensus Target Price from Brokerages
943 Mongodb (NASDAQ:MDB) Given New $226.00 Price Target at Piper Sandler
944 Northern Trust (NASDAQ:NTRS) Price Target Raised to $108.00 at Wells Fargo & Co
945 Iqvia Holdings Inc (NYSE:IQV) Receives $154.75 Consensus Price Target from Brokerages
946 Broadcom (NASDAQ:AVGO) Price Target Increased to $305.00 by Analysts at Cowen
947 Makena Capital Management LLC Buys New Position in Citigroup Inc (NYSE:C)
948 Smith & Nephew (NYSE:SNN) Downgraded by Exane BNP Paribas to Underperform

1685 Stock Traders Purchase High Volume of Adaptimmune Therapeutics Put (NASDAQ:ADAP)
1686 Royal Bank of Canada to Post Q3 2020 of $1.62 Per Share, National Bank Financial Forecasts (TSE:RY)
1687 Companies back ‘black lives  matter’ campaign
1688 First Bancorp Target of Unusually Large Trading (NYSE:FBP)
1689 Channing Capital Management LLC Acquires 10,699 Shares of Citigroup Inc (NYSE:C)
1690 M Holdings Securities Inc. Grows Stock Holdings in Citigroup Inc (NYSE:C)
1691 Channing Capital Management LLC Has $5.32 Million Position in Citigroup Inc (NYSE:C)
1692 Citigroup Inc (NYSE:C) Shares Bought by MBM Wealth Consultants LLC
1693 Ninety One SA PTY Ltd Acquires 2,264 Shares of Citigroup Inc (NYSE:C)
1694 Citigroup chief compliance officer Mark Carawan retires | News | WIN 98.5
1695 Senior Big Data/Java Engineer-AVP (Monitoring Technology) - Technology Shout
1696 South Africa's competition watchdog sues banks accused of forex rigging
1697 Levin Easterly Partners LLC Sells 164,015 Shares 

2597 Workday (NASDAQ:WDAY) Given New $175.00 Price Target at Cowen
2598 Boeing (NYSE:BA) Lifted to Buy at Cfra
2599 Boeing (NYSE:BA) Upgraded by Cfra to “Buy”
2600 Boeing (NYSE:BA) Upgraded by Cfra to “Buy”
2601 Boeing (NYSE:BA) Lifted to Buy at Cfra
2602 Boeing (NYSE:BA) Upgraded by Cfra to “Buy”
2603 Boeing (NYSE:BA) Lifted to Buy at Cfra
2604 Bottomline Technologies (EPAY) Receives a Buy from Barrington
2605 JD.Com (NASDAQ:JD) Shares Up 5.4%
2606 BP (NYSE:BP) Downgraded by Societe Generale to Hold
2607 BP (NYSE:BP) Downgraded by Societe Generale to Hold
2608 BP (NYSE:BP) Downgraded by Societe Generale to Hold
2609 Quinn Opportunity Partners LLC Raises Holdings in Citigroup Inc (NYSE:C)
2610 Citigroup Inc (NYSE:C) Stake Raised by Quinn Opportunity Partners LLC
2611 How to invest globally...during a global pandemic
2612 Royal Bank of Canada (NYSE:RY) Shares Up 8.3%
2613 Autodesk (NASDAQ:ADSK) Price Target Raised to $240.00
2614 Autodesk (NASDAQ:ADSK) Given New $225.00 Price Target at 

3185 B. Riley Increases Madrigal Pharmaceuticals (NASDAQ:MDGL) Price Target to $174.00
3186 Piper Sandler Begins Coverage on EPAM Systems (NYSE:EPAM)
3187 Globant (NYSE:GLOB) Earns Overweight Rating from Analysts at Piper Sandler
3188 Intuit (NASDAQ:INTU) PT Raised to $315.00
3189 Intuit (NASDAQ:INTU) PT Raised to $315.00
3190 Deckers Outdoor (NYSE:DECK) PT Raised to $200.00
3191 Eaton Vance (NYSE:EV) Given New $37.00 Price Target at Cfra
3192 Eversource Energy (NYSE:ES) Now Covered by Seaport Global Securities
3193 Cfra Cuts Eagle Materials (NYSE:EXP) Price Target to $76.00
3194 Cfra Cuts Eagle Materials (NYSE:EXP) Price Target to $76.00
3195 Eaton Vance (NYSE:EV) Price Target Lowered to $37.00 at Cfra
3196 LPL Financial (NASDAQ:LPLA) Price Target Raised to $74.00 at CSFB
3197 Entergy (NYSE:ETR) Research Coverage Started at Seaport Global Securities
3198 TD Ameritrade (NASDAQ:AMTD) Downgraded by Compass Point
3199 Anaplan (NYSE:PLAN) Price Target Raised to $55.00
3200 Mizuho Boosts Ov

3934 Morgan Stanley Trims Public Service Enterprise Group (NYSE:PEG) Target Price to $67.00
3935 Global banks in Hong Kong shorten annual internship programmes, go virtual as coronavirus pandemic upends office life
3936 Homeserve's (HSV) Buy Rating Reiterated at UBS Group
3937 HEXO (NASDAQ:HEXO) Lifted to Buy at Zacks Investment Research
3938 Loop Capital Lowers Cabot Microelectronics (NASDAQ:CCMP) Price Target to $150.00
3939 AstraZeneca AZN) Earns "Overweight" Rating from Barclays
3940 Citigroup Reaffirms Buy Rating for Bank of Georgia Group BGEO)
3941 MacroGenics (NASDAQ:MGNX) Given New $28.00 Price Target at Wedbush
3942 XP Power's (XPP) "Buy" Rating Reiterated at Berenberg Bank
3943 HSBC HSBA) Given a GBX 405 Price Target by UBS Group Analysts
3944 Ferguson FERG) Price Target Raised to GBX 7,000
3945 HSBC Upgrades Kingfisher KGF) to Hold
3946 JPMorgan Chase & Co. Reaffirms Neutral Rating for HENKEL AG & CO/S (OTCMKTS:HENKY)
3947 HENKEL AG & CO/S's (HENKY) Sell Rating Reaffirmed at

### Define distance, index using SimHash, calculate duplicates subset using SimHash only

In [10]:
import logging
logging.getLogger('simhash').setLevel(logging.CRITICAL)

distance = 30
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=distance)

In [11]:
# Select a feed from the list based on its index
SELECTED_INDEX = 6
feed_sel = feeds[SELECTED_INDEX]
print(feed_sel['title'])
# calculate  hash value
feed_hash = Simhash(str(feed_sel['title']))
# find all duplicate indices
dup_indices = index.get_near_dups(feed_hash)
print("Number of duplicates (SimHash): " + str(len(dup_indices)))

Apollo Investment (NASDAQ:AINV) Lifted to Buy at BidaskClub
Number of duplicates (SimHash): 2059


### Print SimHash-based duplicate titles

In [12]:
for dupi in dup_indices:
    print(feeds[int(dupi)]['title'])

Mplx (NYSE:MPLX) Lifted to “Hold” at ValuEngine
Credit Suisse Group Trims Associated British Foods (LON:ABF) Target Price to GBX 2,200
DTE Energy (NYSE:DTE) Raised to "Buy" at Bank of America
Zoom Video Communications (NASDAQ:ZM) Coverage Initiated at Citigroup
Comtech Telecomm. (NASDAQ:CMTL) Posts Quarterly Results, Misses Estimates By $0.22 EPS
Analysts Set Zoetis Inc (NYSE:ZTS) PT at $139.00
Affiliated Managers Group (NYSE:AMG) Stock Rating Lowered by ValuEngine
Essentra's (ESNT) "Buy" Rating Reiterated at Peel Hunt
Q1 2022 Estimate for CubeSmart (NYSE:CUBE) Issued By Jefferies Financial Group
CymaBay Therapeutics (NASDAQ:CBAY) Rating Increased to Buy at Stifel Nicolaus
Bank of Montreal (TSE:BMO) Forecasted to Post Q3 2020 of $0.37 Per Share
Macerich MAC) Price Target Cut to $10.00 by Analysts at Royal Bank of Canada
CNX Midstream Partners (NYSE:CNXM) Trading Down 5.2%
American Electric Power (NYSE:AEP) PT Lowered to $93.00
Veoneer (NYSE:VNE) Raised to “Hold” at Kepler Capital Marke

BidaskClub Lowers Virtu Financial (NASDAQ:VIRT) to Buy
London’s Canary Wharf prepares for return of workers
Wells Fargo & Co Cuts Cooper Companies (NYSE:COO) Price Target to $329.00
Saudi sovereign fund seeks $10 billion margin loan: Bloomberg News
Universal Logistics (NASDAQ:ULH) Upgraded to "Hold" by BidaskClub
TD Ameritrade (NASDAQ:AMTD) Lifted to “Sell” at BidaskClub
China Mengniu Dairy shares soar as it reports sales recovery after worst of the coronavirus as consumers focus on health
United Parcel Service (NYSE:UPS) Rating Reiterated by Wells Fargo & Co
Ciena (NYSE:CIEN) PT Raised to $65.00 at Citigroup
Citigroup Inc (NYSE:C) Shares Acquired by First Interstate Bank
Citigroup Raises Qurate Retail Inc Series A (NASDAQ:QRTEA) Price Target to $12.00
Syndax Pharmaceuticals SNDX) Given "Buy" Rating at Nomura
Meredith (NYSE:MDP) Price Target Increased to $20.00 by Analysts at Citigroup
BidaskClub Downgrades Equinix (NASDAQ:EQIX) to Hold
Amica Retiree Medical Trust Trims Stock Position 

### Calculate and print SimHash + Word2Vec similarity based duplicate titles

In [13]:
count_dup = 0
for dupi in dup_indices:
    try:
        score = calc_similarity(feed_sel['title'], feeds[int(dupi)]['title'], model_w2v_AP)
    except:
        score = 0
    if score > 0.5:
        count_dup += 1
        print(feeds[int(dupi)]['title'])
        print()
print("Number of Dupies (SimHash + WordVectors): " + str(count_dup))

Mplx (NYSE:MPLX) Lifted to “Hold” at ValuEngine

Affiliated Managers Group (NYSE:AMG) Stock Rating Lowered by ValuEngine

CymaBay Therapeutics (NASDAQ:CBAY) Rating Increased to Buy at Stifel Nicolaus

Veoneer (NYSE:VNE) Raised to “Hold” at Kepler Capital Markets

CNX Midstream Partners LP (NYSE:CNXM) Receives $12.00 Consensus PT from Brokerages

Analysts Set Xylem Inc (NYSE:XYL) Target Price at $68.62

Wedbush Boosts Paypal (NASDAQ:PYPL) Price Target to $176.00

BP BP) Downgraded by Morgan Stanley

UBS Group Reaffirms Buy Rating for Prudential (LON:PRU)

Kubota (OTCMKTS:KUBTY) Downgraded to Hold at Zacks Investment Research

BidaskClub Lowers Facebook (NASDAQ:FB) to Buy

Ship Finance International (NYSE:SFL) Lifted to Buy at Zacks Investment Research

Moffett Nathanson Reaffirms Buy Rating for Paypal (NASDAQ:PYPL)

Boeing (NYSE:BA) Lifted to Buy at Cfra

Deutsche Bank Boosts Docusign (NASDAQ:DOCU) Price Target to $160.00

Wells Fargo & Co Reaffirms Sell Rating for DXC Technology (NYSE:

Citigroup Raises Chart Industries (NASDAQ:GTLS) Price Target to $50.00

Huaneng Power International Inc ADR Class N (NYSE:HNP) Upgraded by Zacks Investment Research to Buy

Tigress Financial Reiterates Buy Rating for Herbalife Nutrition (NYSE:HLF)

Blackstone Mortgage Trust Announces Public Offering Of Class A Common Stock

PhaseBio Pharmaceuticals Inc (NASDAQ:PHAS) Short Interest Down 10.1% in April

Zacks Investment Research Lowers Huaneng Power International Inc ADR Class N (NYSE:HNP) to Hold

Advanced Energy Industries (NASDAQ:AEIS) Upgraded to "Buy" at BidaskClub

Apollo Investment (NASDAQ:AINV) Rating Increased to Hold at BidaskClub

Adaptimmune Therapeutics (NASDAQ:ADAP) Upgraded to Buy by Zacks Investment Research

BidaskClub Upgrades Popular (NASDAQ:BPOP) to Hold

Baidu (NASDAQ:BIDU) Rating Increased to Sell at BidaskClub

Bank of Montreal (TSE:BMO) Cut to Sector Perform at National Bank Financial

VICI Properties (NYSE:VICI) Raised to “Buy” at Zacks Investment Research

Comme

Cetera Advisors LLC Acquires 236 Shares of Citigroup Inc (NYSE:C)

HENKEL AG & CO/S (OTCMKTS:HENKY) Earns Underweight Rating from Morgan Stanley

Micro Focus International’s (MCRO) “Neutral” Rating Reiterated at UBS Group

Discovery Communications Inc. (NASDAQ:DISCA) Receives Consensus Recommendation of “Hold” from Analysts

Citigroup Inc (NYSE:C) Position Lifted by Cigna Investments Inc. New

GRUPO AVAL ACCI/S (NYSE:AVAL) Downgraded by Citigroup

NexTier Oilfield Solutions (NYSE:NEX) Raised to "Buy" at Zacks Investment Research

Oppenheimer Reiterates Outperform Rating for Nasdaq (NASDAQ:NDAQ)

Analysts Set ANSYS, Inc. (NASDAQ:ANSS) Target Price at $260.11

Citigroup Inc (NYSE:C) Shares Sold by Wealth Quarterback LLC

Buzzi Unicem (OTCMKTS:BZZUF) Lifted to “Strong-Buy” at Zacks Investment Research

Apple (NASDAQ:AAPL) Reaches New 12-Month High on Analyst Upgrade

Analysts Set RenaissanceRe Holdings Ltd. (NYSE:RNR) Target Price at $188.50

Adaptimmune Therapeutics ADAP) Cut to Buy at B

Apollo Global Management (NYSE:APO) Downgraded by Citigroup

TCG BDC (NASDAQ:CGBD) Downgraded by Zacks Investment Research

CubeSmart (NYSE:CUBE) Downgraded to Sell at Zacks Investment Research

Nabors Industries (NYSE:NBR) Downgraded by ValuEngine to Buy

Ardelyx (NASDAQ:ARDX) Rating Lowered to Hold at BidaskClub

Childrens Place (NASDAQ:PLCE) Stock Rating Lowered by TheStreet

Jefferies Financial Group Boosts Britvic (LON:BVIC) Price Target to GBX 850

Juan Colombás Acquires 339 Shares of Lloyds Banking Group PLC (LON:LLOY) Stock

Britvic’s (BVIC) Equal Weight Rating Reiterated at Barclays

Endo International (NASDAQ:ENDP) Cut to Strong Sell at BidaskClub

PVH Corp PVH) Sees Large Growth in Short Interest

Citigroup Inc (NYSE:C) Shares Sold by Asset Management One Co. Ltd.

GreenSky (NASDAQ:GSKY) Lifted to “Hold” at BidaskClub

Investors Buy Large Volume of Apollo Investment Put Options (NASDAQ:AINV)

Pinduoduo (NASDAQ:PDD) Rating Lowered to Hold at Jefferies Financial Group

Traders